### Main File for our testing for the project


In [1]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import csv
# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#Keras
import keras
import tensorflow as tf
from keras.utils import to_categorical


# Testing code from another strategy

In [2]:
# Import the dataset
img_height = 288
img_width = 432
batch_size = 10
data_dir = "../data/Images_Original"

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    labels = "inferred",
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    shuffle = True,
    batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    labels = "inferred",
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    shuffle = True,
    batch_size=batch_size)


Found 999 files belonging to 10 classes.
Using 800 files for training.
Found 999 files belonging to 10 classes.
Using 199 files for validation.


# Recent Strategy

# Let's add some functions to generate the full spectrogram and generate segments of the spectrogram so we have more data to train on.

In [3]:
def get_melspectrogram(wav_file_path, length=30, duration_of_segments=5 ):
    """
    Get mel spectrogram for a given wav file and divide it into parts.

    :param wav_file_path: Path to the source wav file
    :param length: length in seconds of the source audio file. Defaults to 30.
    :param duration_of_segments: duration of segments in seconds. number of segments = length/duration_of_segments. Defaults to 5.
    :return: Mel spectrogram of the source wav file. Segments will be saved to a file and it's path will be printed.
    """
    y, sr = librosa.load(wav_file_path, sr=None, duration=length)
    melspectrogram = librosa.feature.melspectrogram(y=y, sr=sr)

    # Determine the number of samples in the duration of segments
    samples_per_segment = sr * duration_of_segments

    # Number of segments
    num_segments=int(length/duration_of_segments)

    # Loop through the audio signal and extract the segments
    for i in range(num_segments):
        # Get the start and end indices of the segment
        start = i * samples_per_segment
        end = start + samples_per_segment

        # Extract the segment from the audio signal
        segment = y[start:end]

        # Compute the mel spectrogram of the segment
        mel_spec_segment = librosa.feature.melspectrogram(y=segment, sr=sr)

        # sample name
        sample_name = wav_file_path.replace("../data/genres_original/","").replace(".wav","")

        # path to save samples to
        sample_name = f'{sample_name.split("/")[0]}/npy/{sample_name.split("/")[1]}'

        save_path = f'../data/mel_spec_samples/{sample_name}_{i}.npy'
        # Save the mel spectrogram to a file
        np.save(save_path, mel_spec_segment)
        print(f'Saved segment /{sample_name}_{i}.npy')

    return melspectrogram


def plot_melspectrogram(melspectrogram):
    """
    Plot mel spectrogram using pyplot to visualize the data. Works for both full spectrogram and segments.

    :param melspectrogram: np.array of mel spectrogram generated using librosa.feature.melspectrogram()
    :return: void
    """
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(librosa.power_to_db(melspectrogram, ref=np.max), y_axis='mel', fmax=8000, x_axis='time')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Mel spectrogram')
    plt.tight_layout()
    plt.show()


In [4]:
# test segmentation of audio file
genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']
for test_genre in genres:
    # Make the directories to avoid file not found errors
    os.makedirs(f'../data/mel_spec_samples/{test_genre}', exist_ok=True)
    os.makedirs(f'../data/mel_spec_samples/{test_genre}/npy', exist_ok=True)
    os.makedirs(f'../data/mel_spec_samples/{test_genre}/png', exist_ok=True)

    # Filename to test functions
    for i in range(0,10):
        sample_number = ""
        if i < 10:
            sample_number = f'0{i}'
        else:
            sample_number = f'{i}'
        test_filename = f'../data/genres_original/{test_genre}/{test_genre}.000{sample_number}.wav'

        # Test the main get_melspectrogram function to generate the full audio spectrogram of a given file, and save segments of itself to files
        mel_spectrogram = get_melspectrogram(test_filename, length=30, duration_of_segments=5)





Saved segment /blues/npy/blues.00000_0.npy
Saved segment /blues/npy/blues.00000_1.npy
Saved segment /blues/npy/blues.00000_2.npy
Saved segment /blues/npy/blues.00000_3.npy
Saved segment /blues/npy/blues.00000_4.npy
Saved segment /blues/npy/blues.00000_5.npy
Saved segment /blues/npy/blues.00001_0.npy
Saved segment /blues/npy/blues.00001_1.npy
Saved segment /blues/npy/blues.00001_2.npy
Saved segment /blues/npy/blues.00001_3.npy
Saved segment /blues/npy/blues.00001_4.npy
Saved segment /blues/npy/blues.00001_5.npy
Saved segment /blues/npy/blues.00002_0.npy
Saved segment /blues/npy/blues.00002_1.npy
Saved segment /blues/npy/blues.00002_2.npy
Saved segment /blues/npy/blues.00002_3.npy
Saved segment /blues/npy/blues.00002_4.npy
Saved segment /blues/npy/blues.00002_5.npy
Saved segment /blues/npy/blues.00003_0.npy
Saved segment /blues/npy/blues.00003_1.npy
Saved segment /blues/npy/blues.00003_2.npy
Saved segment /blues/npy/blues.00003_3.npy
Saved segment /blues/npy/blues.00003_4.npy
Saved segme

# Run this if you are getting 'Directory Not Found' errors

In [5]:
# Run this if you are getting 'Directory Not Found' errors

genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']
for genre in genres:
    os.makedirs(f'../data/mel_spec_samples/{genre}', exist_ok=True)
    os.makedirs(f'../data/mel_spec_samples/{genre}/npy', exist_ok=True)
    os.makedirs(f'../data/mel_spec_samples/{genre}/png', exist_ok=True)

print('done')

done


# Now let's generate the images from the mel spectrogram segments we generated above.

In [6]:
# Function to generate images from mel spectrograms for a given genre
def generate_images_for_genre(genre, num_samples=6):
    """
    Generate images from mel spectrogram segments for a given genre and save them to the same directory as the segments.

    :param genre: string of genre to generate images for
    :param num_samples: number of samples. Defaults to 6.
    :return: False if error, True if successful
    """
    genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']
    if genre not in genres:
        print(f'Genre \"{genre}\" not in list of genres')
        return False
    for j in range(0,100):
        sample_number = ""
        if j < 10:
            sample_number = f'0{j}'
        else:
            sample_number = f'{j}'

        for i in range(num_samples):
            mel_segment = np.load(f'../data/mel_spec_samples/{genre}/npy/{genre}.000{sample_number}_{i}.npy')
            plt.figure(figsize=(10, 4))
            librosa.display.specshow(librosa.power_to_db(mel_segment, ref=np.max), y_axis='mel', fmax=8000, x_axis='time')
            plt.colorbar(format='%+2.0f dB')
            plt.title(f'{genre}.00000_{i}.npy')
            plt.tight_layout()
            save_name = f'../data/mel_spec_samples/{genre}/png/{genre}.000{sample_number}_{i}.png'
            plt.savefig(save_name, bbox_inches='tight', pad_inches=0)
            plt.close()
            print(f'Saved image: {save_name}')

    return True

### Test that bad boy

#### This will take a minute or two to run

In [7]:
generate_images_for_genre('classical', num_samples=5)

Saved image: ../data/mel_spec_samples/classical/png/classical.00000_0.png
Saved image: ../data/mel_spec_samples/classical/png/classical.00000_1.png
Saved image: ../data/mel_spec_samples/classical/png/classical.00000_2.png
Saved image: ../data/mel_spec_samples/classical/png/classical.00000_3.png
Saved image: ../data/mel_spec_samples/classical/png/classical.00000_4.png
Saved image: ../data/mel_spec_samples/classical/png/classical.00001_0.png
Saved image: ../data/mel_spec_samples/classical/png/classical.00001_1.png
Saved image: ../data/mel_spec_samples/classical/png/classical.00001_2.png
Saved image: ../data/mel_spec_samples/classical/png/classical.00001_3.png
Saved image: ../data/mel_spec_samples/classical/png/classical.00001_4.png
Saved image: ../data/mel_spec_samples/classical/png/classical.00002_0.png
Saved image: ../data/mel_spec_samples/classical/png/classical.00002_1.png
Saved image: ../data/mel_spec_samples/classical/png/classical.00002_2.png
Saved image: ../data/mel_spec_samples/

KeyboardInterrupt: 

## Now let's generate images for all genre samples

#### This will take a while to run

In [113]:
# test segmentation of audio file
genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']

for test_genre in genres:

    for i in range(0,100):
        sample_number = ""
        if i < 10:
            sample_number = f'0{i}'
        else:
            sample_number = f'{i}'



        test_filename = f'../data/genres_original/{test_genre}/{test_genre}.000{sample_number}.wav'


        # Test the main get_melspectrogram function to generate the full audio spectrogram of a given file, and save segments of itself to files
        mel_spectrogram = get_melspectrogram(test_filename, length=30, duration_of_segments=5)

        # Generate images for all the genre segments
        # COMMENTED CUS WE DONT NEED RN
        # generate_images_for_genre(test_genre, num_samples=6)


Saved segment /classical/npy/classical.00000_0.npy
Saved segment /classical/npy/classical.00000_1.npy
Saved segment /classical/npy/classical.00000_2.npy
Saved segment /classical/npy/classical.00000_3.npy
Saved segment /classical/npy/classical.00000_4.npy
Saved segment /classical/npy/classical.00000_5.npy
Saved segment /classical/npy/classical.00001_0.npy
Saved segment /classical/npy/classical.00001_1.npy
Saved segment /classical/npy/classical.00001_2.npy
Saved segment /classical/npy/classical.00001_3.npy
Saved segment /classical/npy/classical.00001_4.npy
Saved segment /classical/npy/classical.00001_5.npy
Saved segment /classical/npy/classical.00002_0.npy
Saved segment /classical/npy/classical.00002_1.npy
Saved segment /classical/npy/classical.00002_2.npy
Saved segment /classical/npy/classical.00002_3.npy
Saved segment /classical/npy/classical.00002_4.npy
Saved segment /classical/npy/classical.00002_5.npy
Saved segment /classical/npy/classical.00003_0.npy
Saved segment /classical/npy/cl

In [11]:
# normalize the mel spectrogram values to have zero mean and unit variance
def normalize_melspectrogram(mel_spec_path):
    mel_spec = np.load(mel_spec_path)

    # Reshape the data to have a channel dimension of 1
    mel_spec = mel_spec.reshape(mel_spec.shape[0], mel_spec.shape[1], 1)

    scaler = StandardScaler()
    mel_spec_normalized = scaler.fit_transform(mel_spec.reshape(mel_spec.shape[0], -1))
    mel_spec_normalized = mel_spec_normalized.reshape(mel_spec.shape)
    return mel_spec_normalized


In [241]:
def get_features_and_labels():
    genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']

    features = []
    labels = []

    print(f'Extracting...')

    for genre in genres:
        count = 0
        for file in os.listdir(f'../data/mel_spec_samples/{genre}/npy'):

            # Extracting Mel Spectrogram feature
            # Use normalize_melspectrogram to get normalized mel spectrogram features to fit model
            melspectrogram = normalize_melspectrogram(f'../data/mel_spec_samples/{genre}/npy/{file}')
            spec_mean = melspectrogram.mean(axis=1)
            spec_min = melspectrogram.min(axis=1)
            spec_max = melspectrogram.max(axis=1)
            spec_feature = np.concatenate( (spec_mean, spec_min, spec_max) )

            # Extracting Label
            label = genres.index(genre)

            # Appending features and labels
            features.append(melspectrogram)
            labels.append(label)
            count += 1

        print(f'Extracted {count} features and labels for genre: {genre}')


    print('Finished extracting features and labels for all genres')
    return features, labels


def split_data(features, labels, testSize=0.2, randomState=42):
    # Splitting the data into training, testing, and validation sets
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=testSize, random_state=randomState)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=testSize, random_state=randomState)



    # Converting the labels to categorical
    y_train = to_categorical(y_train, num_classes=10)
    y_test = to_categorical(y_test, num_classes=10)
    y_val = to_categorical(y_val, num_classes=10)


    print(f'len of X_train: {len(X_train)} \n len of y_train: {len(y_train)}')
    print(f'len of X_test: {len(X_test)} \n len of y_test: {len(y_test)}')
    print(f'len of X_val: {len(X_val)} \n len of y_val: {len(y_val)}')

    return X_train, X_val, X_test, y_train, y_val, y_test


In [229]:
# Get features and labels
features, labels = get_features_and_labels()

Extracting...
Extracted 600 features and labels for genre: blues
Extracted 600 features and labels for genre: classical
Extracted 600 features and labels for genre: country
Extracted 600 features and labels for genre: disco
Extracted 600 features and labels for genre: hiphop
Extracted 600 features and labels for genre: jazz
Extracted 600 features and labels for genre: metal
Extracted 600 features and labels for genre: pop
Extracted 600 features and labels for genre: reggae
Extracted 600 features and labels for genre: rock
Finished extracting features and labels for all genres


In [242]:
X_train, X_val, X_test, y_train, y_val, y_test = split_data(features, labels, testSize=0.2, randomState=42)



print('\n\n\n\n\n')
print(f'x_train: {X_train[0].shape}')
print(f'y_train: {y_train[0].shape}')
print(f'x_val: {X_val[0].shape}')
print(f'y_val: {y_val[0].shape}')
print(f'x_test: {X_test[0].shape}')
print(f'y_test: {y_test[0].shape}')


len of X_train: 3840 
 len of y_train: 3840
len of X_test: 1200 
 len of y_test: 1200
len of X_val: 960 
 len of y_val: 960






x_train: (128, 216, 1)
y_train: (10,)
x_val: (128, 216, 1)
y_val: (10,)
x_test: (128, 216, 1)
y_test: (10,)


In [227]:
# Split data into training, testing, and validation sets


percent_sixty = int(len(features) * 0.6)
percent_seventy = int(len(features) * 0.7)
percent_eighty = int(len(features) * 0.8)
percent_hundred = int(len(features) * 1.0)

# First 60% of features and labels are training data
X_train = features[0:percent_sixty]
y_train = labels[0:percent_sixty]
# Next 20% of features and labels are validation data
X_val = features[percent_sixty:percent_eighty]
y_val = labels[percent_sixty:percent_eighty]
# Last 20% of features and labels are testing data
X_test = features[percent_eighty:percent_hundred]
y_test = labels[percent_eighty:percent_hundred]

X_train, X_val, X_test, y_train, y_val, y_test = split_data(features, labels, testSize=0.2, randomState=42)

print(f'x_train: {len(X_train)}')
print(f'y_train: {len(y_train)}')
print(f'x_val: {len(X_val)}')
print(f'y_val: {len(y_val)}')
print(f'x_test: {len(X_test)}')
print(f'y_test: {len(y_test)}')

print(f'x_train: {X_train[0].shape}')
print(f'y_train: {y_train[0].shape}')
print(f'x_val: {X_val[0].shape}')
print(f'y_val: {y_val[0].shape}')
print(f'x_test: {X_test[0].shape}')
print(f'y_test: {y_test[0].shape}')


len of X_train: 3840 
 len of y_train: 3840
len of X_test: 1200 
 len of y_test: 1200
len of X_val: 960 
 len of y_val: 960
x_train: 3840
y_train: 3840
x_val: 960
y_val: 960
x_test: 1200
y_test: 1200
x_train: (384,)
y_train: (10,)
x_val: (384,)
y_val: (10,)
x_test: (384,)
y_test: (10,)


In [243]:
# Basic network design

inputs = keras.Input(shape=(216,), name="feature")
x = keras.layers.Dense(300, activation="relu", name="dense_1")(inputs)
x = keras.layers.Dense(200, activation="relu", name="dense_2")(x)
outputs = keras.layers.Dense(3, activation="softmax", name="predictions")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    # Optimizer
    optimizer=keras.optimizers.RMSprop(),
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [135]:


y_train.shape

ValueError: cannot reshape array of size 499200 into shape (128,10)

In [244]:
# Train the model
print(f'x_train: {X_train[0].shape}')
print(f'y_train: {y_train[0].shape}')

model.fit(x=X_train, y=y_train,batch_size=32, verbose=1,  epochs=64)

x_train: (128, 216, 1)
y_train: (10,)


ValueError: Data cardinality is ambiguous:
  x sizes: 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 1

# See how it went broski

In [ ]:
score = model.evaluate(x=X_test,y=y_test, verbose=0)
print('Accuracy : ' + str(score[1]*100) + '%')

# Normalize the Data so that it has 0 mean and unit variance

In [ ]:
# Test that each genre is generating spectrograms correctly (TEST CODE)

In [ ]:
# load saved spectrogram from .npy files and plot them in subplots
test_filename = f'../data/mel_spec_samples/blues/blues.00000_0.npy'
var = np.load(test_filename)

plot_melspectrogram(var)

# Generate normalized mel spectrogram
mel_spectrograms_normalized = normalize_melspectrogram('../data/mel_spec_samples/blues/blues.00000_0.npy')

print(f'shape: {mel_spectrograms_normalized.shape}')
# Plot the first mel spectrogram in the normalized data as a line plot
plt.plot(mel_spectrograms_normalized[0].flatten())
plt.title('Normalized Mel Spectrogram')
plt.xlabel('Mel spectrogram bin')
plt.ylabel('Normalized value')
plt.show()

# Plot the first mel spectrogram in the normalized data as a line plot
plt.plot(var.flatten())
plt.title('Original Mel Spectrogram')
plt.xlabel('Mel spectrogram bin')
plt.ylabel('Normalized value')
plt.show()



